In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import opendatasets as od
import tensorflow as tf
import cv2
from tqdm import *

In [2]:
dataset_directory = "C:\\Users\\amith\\Documents\\Datasets\\kaggle\\kaggle".replace("\\","/")
csv_directory = "C:\\Users\\amith\\Documents\\Datasets".replace("\\","/")

In [3]:
train_val_age = pd.read_csv(csv_directory + "/train_age.csv").sort_values(by="imageId")
train_val_gender = pd.read_csv(csv_directory + "/train_gender.csv").sort_values(by="imageId")

id = train_val_age["imageId"].values.tolist()
train_val_files = pd.DataFrame(columns=["imageId","Files"])
ctr = 0

for i in os.listdir(dataset_directory+"/train"):
    train_val_files.loc[ctr,"imageId"] = id[ctr]
    train_val_files.loc[ctr,"Files"] = i
    ctr += 1

train_val_files = train_val_files.sort_values(by="imageId")

train_val_output = pd.merge(left=train_val_age,right=train_val_gender,on="imageId",how="inner")
train_val_output = pd.merge(left=train_val_output,right=train_val_files,on="imageId",how="inner")

train_val_output = train_val_output.sort_values(by="imageId")

train_val_id = train_val_output["imageId"].values.tolist()
np.random.shuffle(train_val_id)


threshold = int(np.round(0.8 * len(train_val_id)))

train_id = train_val_id[0:threshold]
val_id = train_val_id[threshold:]

train_val_output.set_index(keys="imageId",drop=False,inplace=True)

train_output = train_val_output.loc[train_id,:]
val_output = train_val_output.loc[val_id,:]

train_output = train_output.reset_index(drop=True).drop(columns=["imageId"])
val_output = val_output.reset_index(drop=True).drop(columns=["imageId"])

In [4]:
train_output.head()

,age,gender,Files
0,20.0,0,009458.png
1,34.0,1,006451.png
2,30.0,0,006159.png
3,21.0,1,002542.png
4,81.0,0,010648.png


In [5]:
val_output.head()

,age,gender,Files
0,46.0,1,002837.png
1,41.0,0,006465.png
2,43.0,0,008600.png
3,43.0,0,009545.png
4,57.0,0,003439.png


In [6]:
train_images = []
train_data = {}
ctr = 0
for i in tqdm(iterable=train_output["Files"],desc="Processing train data"):
    ages = train_output.loc[ctr,"age"]
    genders = train_output.loc[ctr,"gender"]
    img_org = cv2.imread(dataset_directory + "/train/" + i)
    img_gray = cv2.cvtColor(src=img_org, code=cv2.COLOR_BGR2GRAY)
    img_short = cv2.resize(src=img_gray, dsize=(64,64))
    img = img_short / 255.0
    train_images.append(img)
    ctr += 1

train_images = np.array(train_images)

Processing train data: 100%|██████████| 8562/8562 [03:18<00:00, 43.12it/s]


In [7]:
dataset_directory + "/" + i

'C:/Users/amith/Documents/Datasets/kaggle/kaggle/007907.png'

In [8]:
dataset_directory + "/val/" + i

'C:/Users/amith/Documents/Datasets/kaggle/kaggle/val/007907.png'

In [9]:
val_images = []
val_data = {}
ctr = 0
for i in tqdm(iterable=val_output["Files"],desc="Processing val data"):
    ages = val_output.loc[ctr,"age"]
    genders = val_output.loc[ctr,"gender"]
    img_org = cv2.imread(dataset_directory + "/train/" + i)
    img_gray = cv2.cvtColor(src=img_org, code=cv2.COLOR_BGR2GRAY)
    img_short = cv2.resize(src=img_gray, dsize=(64,64))
    img = img_short / 255.0
    val_images.append(img)
    ctr += 1

val_images = np.array(val_images)

Processing val data: 100%|██████████| 2140/2140 [00:40<00:00, 53.11it/s]


In [10]:
train_images = train_images.reshape(tuple(list(train_images.shape) + [1]))
val_images = val_images.reshape(tuple(list(val_images.shape) + [1]))

In [11]:
val_images.shape

(2140, 64, 64, 1)

In [12]:
model_genders = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64,activation="relu"),
    tf.keras.layers.Dense(units=1,activation="softmax")
])

In [13]:
model_ages = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64,activation="relu"),
    tf.keras.layers.Dense(units=1,activation="relu")
])

In [14]:
train_ages = train_output["age"].values
train_genders = train_output["gender"].values

val_ages = val_output["age"].values
val_genders = val_output["gender"].values

In [15]:
model_genders.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.binary_crossentropy,metrics=['accuracy'])
model_ages.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.mean_absolute_error, metrics=["mae"])

In [16]:
model_genders.fit(x=train_images,y=train_genders,batch_size=32,epochs=10,validation_data=(val_images,val_genders),verbose=1,shuffle=True,callbacks=tf.keras.callbacks.EarlyStopping(patience=2))

Epoch 1/10
268/268 [==============================] - 46s 166ms/step - loss: 0.6094 - accuracy: 0.4179 - val_loss: 0.5360 - val_accuracy: 0.4276
Epoch 2/10
268/268 [==============================] - 42s 158ms/step - loss: 0.4925 - accuracy: 0.4179 - val_loss: 0.6023 - val_accuracy: 0.4276
Epoch 3/10
268/268 [==============================] - 45s 168ms/step - loss: 0.4466 - accuracy: 0.4179 - val_loss: 0.4202 - val_accuracy: 0.4276
Epoch 4/10
268/268 [==============================] - 52s 195ms/step - loss: 0.3781 - accuracy: 0.4179 - val_loss: 0.4254 - val_accuracy: 0.4276
Epoch 5/10
268/268 [==============================] - 45s 167ms/step - loss: 0.3511 - accuracy: 0.4179 - val_loss: 0.3387 - val_accuracy: 0.4276
Epoch 6/10
268/268 [==============================] - 44s 165ms/step - loss: 0.3153 - accuracy: 0.4179 - val_loss: 0.3448 - val_accuracy: 0.4276
Epoch 7/10
268/268 [==============================] - 47s 176ms/step - loss: 0.2921 - accuracy: 0.4179 - val_loss: 0.3471 - val_ac

In [17]:
model_ages.fit(x=train_images,y=train_ages,batch_size=32,epochs=10,validation_data=(val_images,val_ages),verbose=1,shuffle=True,callbacks=tf.keras.callbacks.EarlyStopping(patience=2))

Epoch 1/10
268/268 [==============================] - 45s 160ms/step - loss: 15.1653 - mae: 15.1653 - val_loss: 12.9404 - val_mae: 12.9404
Epoch 2/10
268/268 [==============================] - 45s 166ms/step - loss: 11.5321 - mae: 11.5321 - val_loss: 11.5185 - val_mae: 11.5185
Epoch 3/10
268/268 [==============================] - 45s 169ms/step - loss: 10.7763 - mae: 10.7763 - val_loss: 10.5220 - val_mae: 10.5220
Epoch 4/10
268/268 [==============================] - 52s 196ms/step - loss: 9.8165 - mae: 9.8165 - val_loss: 10.3268 - val_mae: 10.3268
Epoch 5/10
268/268 [==============================] - 54s 201ms/step - loss: 9.7416 - mae: 9.7416 - val_loss: 9.5323 - val_mae: 9.5323
Epoch 6/10
268/268 [==============================] - 56s 206ms/step - loss: 9.1173 - mae: 9.1173 - val_loss: 10.0309 - val_mae: 10.0309
Epoch 7/10
268/268 [==============================] - 49s 184ms/step - loss: 8.9402 - mae: 8.9402 - val_loss: 9.1284 - val_mae: 9.1284
Epoch 8/10
268/268 [===================

In [18]:
tf.keras.models.save_model(model=model_genders,filepath="C:/Users/amith/Documents/Datasets/model_genders.h5",overwrite=True)
tf.keras.models.save_model(model=model_ages,filepath="C:/Users/amith/Documents/Datasets/model_ages.h5",overwrite=True)

C:\Users\amith\AppData\Local\Temp\ipykernel_14736\4190216403.py:1: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model=model_genders,filepath="C:/Users/amith/Documents/Datasets/model_genders.h5",overwrite=True)
C:\Users\amith\AppData\Local\Temp\ipykernel_14736\4190216403.py:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model=model_ages,filepath="C:/Users/amith/Documents/Datasets/model_ages.h5",overwrite=True)


In [19]:
print(dataset_directory + "/test/")

C:/Users/amith/Documents/Datasets/kaggle/kaggle/test/


In [20]:
test_images = []

ctr = 0
for i in tqdm(iterable=sorted(os.listdir(dataset_directory + "/test/")),desc="Processing test data"):
    img_org = cv2.imread(dataset_directory + "/test/" + i)
    img_gray = cv2.cvtColor(src=img_org, code=cv2.COLOR_BGR2GRAY)
    img_short = cv2.resize(src=img_gray, dsize=(64,64))
    img = img_short / 255.0
    test_images.append(img)
    ctr += 1

test_images = np.array(test_images)

Processing test data: 100%|██████████| 11747/11747 [03:59<00:00, 48.96it/s]


In [21]:
test_ages = pd.read_csv("C:\\Users\\amith\\Documents\\Datasets\\sample_submission_age.csv")["age"].copy()
test_genders = pd.read_csv("C:\\Users\\amith\\Documents\\Datasets\\sample_submission_gender.csv")["gender"].copy()

In [22]:
predict_ages = model_ages.predict(test_images)

368/368 [==============================] - 19s 50ms/step


In [23]:
predict_genders = model_genders.predict(test_images)

368/368 [==============================] - 18s 47ms/step


In [24]:
predict_genders

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [25]:
df_pred_age = pd.DataFrame(columns=["imageId","age"])
df_pred_gender = pd.DataFrame(columns=["imageId","gender"])

df_pred_age["imageId"] = list(range(predict_ages.shape[0]))
df_pred_age["age"] = np.int64(np.round(predict_ages,0))

df_pred_gender["imageId"] = list(range(predict_genders.shape[0]))
df_pred_gender["gender"] = np.int64(np.round(predict_genders,0))

In [26]:
df_pred_age

,imageId,age
0,0,54
1,1,33
2,2,39
3,3,63
4,4,40
...,...,...
11742,11742,65
11743,11743,43
11744,11744,57
11745,11745,48


In [27]:
df_pred_gender

,imageId,gender
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
11742,11742,1
11743,11743,1
11744,11744,1
11745,11745,1


In [28]:
df_pred_gender.to_csv(path_or_buf="C:/Users/amith/Documents/Datasets/Saved/sample_submission_gender.csv",index=False)
df_pred_age.to_csv(path_or_buf="C:/Users/amith/Documents/Datasets/Saved/sample_submission_age.csv",index=False)